In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, date_format, concat_ws, to_timestamp, to_date, expr, StringType, collect_list, udf
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("FB").getOrCreate()

df = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0.csv", header=True, inferSchema=True)
df2 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (1).csv", header=True, inferSchema=True)
df3 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (2).csv", header=True, inferSchema=True)
df4 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (3).csv", header=True, inferSchema=True)


In [12]:
df.show(5)

+---+----------+-------------------+-----------+-------------+----+----+---+----+----+---+---------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+----+----+-----+----+---+----+----+----+----+----+----+----+----+----+-----+--------+--------+-----+-----+-------+-------+-------+-------+-----+-------+-------+----+----+------+------+------+------+------+------+------+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+---------+---------+------+------+--------+--------+--------+--------+-----+--------+--------+-----+-----+-------+-------+-------+-------+
|Div|      Date|               Time|   HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|  Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA| PSH| PSD|  PSA| WHH|WHD| WHA| VCH| VCD| VCA|MaxH|MaxD|MaxA|AvgH|AvgD| AvgA|B365>2.5|B365<2.5|P>2.5|P<2.5|Max>2.5|Max<2.5|Avg>2.5|Avg<2.5|

In [13]:
df = df.union(df2)
df = df.union(df3)
df = df.union(df4)


In [14]:
df = df.drop("Div")
df = df.select("Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR")
df.groupBy("Referee").count().orderBy("count", ascending=False).show(300)

+------------+-----+
|     Referee|count|
+------------+-----+
|    A Taylor|  113|
|    M Oliver|  108|
|   P Tierney|  103|
|    C Pawson|   89|
|   S Attwell|   85|
|    S Hooper|   81|
|    A Madley|   79|
|     D Coote|   78|
|  C Kavanagh|   73|
|     R Jones|   69|
|    P Bankes|   59|
|  A Marriner|   57|
|   D England|   56|
|  M Atkinson|   52|
|      M Dean|   51|
|      J Moss|   49|
|   J Gillett|   46|
|    J Brooks|   45|
|    K Friend|   43|
|     G Scott|   37|
| M Salisbury|   32|
|  T Robinson|   23|
|   T Bramall|   19|
|T Harrington|   19|
|   S Barrott|   15|
|     L Mason|   11|
|      D Bond|    9|
|     J Smith|    5|
|   S Allison|    3|
|    R Madley|    2|
|     R Welch|    2|
|     L Smith|    2|
|   M Donohue|    2|
|  J Gillett |    1|
|     S Singh|    1|
|      A Moss|    1|
+------------+-----+



In [15]:
df = df.withColumn("corrected_date", expr("concat('20', substr(date, 9, 2), '-', substr(date, 4, 2), '-', substr(date, 1, 2))"))
df = df.withColumn("corrected_date_type", to_date(col("corrected_date")))
df = df.withColumn("updated_datetime", (to_timestamp(concat_ws(' ', date_format(col("corrected_date_type"), 'yyyy-MM-dd'), date_format(col("Time"), 'HH:mm:ss')))))
df = df.drop("Date", "Time", "corrected_date")
df = df.withColumnRenamed("corrected_date_type", "Date")
df = df.withColumnRenamed("updated_datetime", "Time")


In [16]:
#Last 5 game results

window_spec = Window.partitionBy("HomeTeam").orderBy("Time")

df = df.withColumn("last_result", lag("FTR", 5).over(window_spec))

def list_to_string(results):
    return udf(''.join(results[-5:]))

last_5_results = df.groupBy("HomeTeam").agg(collect_list("FTR").alias("last_5_results"))

list_to_string_udf = udf(list_to_string, StringType())

df = last_5_results.withColumn("last_5_results_str", list_to_string_udf("last_5_results"))
df = df.drop("last_5_results", "element")

df.printSchema()


root
 |-- HomeTeam: string (nullable = true)
 |-- last_5_results_str: string (nullable = true)



In [17]:
df.head(5)

Py4JJavaError: An error occurred while calling o311.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 42) (DESKTOP-KPGKSR4 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun